In [1]:
import os
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#from google.colab import files
#uploaded = files.upload()
df = pd.read_csv('full_dataset_filtered.csv')

In [5]:
# DON'T USE IT
# This method segments all the data into ordinal epochs.
def segment_epochs_all(data):
    data['signal'] = data['trigger'].apply(lambda x: 'Yes' if x == 1 or x == -1 else 'No')

    data['epoch'] = 0
    # Calculating time difference between samples
    fs = 250  # Sampling frequency in Hz
    time_diff = 1 / fs  # Time difference between samples

    # Time before and after trigger event (in seconds)
    time_before = 0.1
    time_after = 0.7

    # Number of samples before and after the trigger event
    n_before = int(time_before / time_diff)  # Should be 25 samples
    n_after = int(time_after / time_diff)    # Should be 175 samples

    # Total number of samples in an epoch
    n_epoch = n_before + n_after  # 200 samples

    # Get the unique participant IDs
    participant_ids = data['participant_id'].unique()

    for participant_id in participant_ids:
        # Filter data for the current participant
        participant_data = data[data['participant_id'] == participant_id].reset_index(drop=True)

        # Reset index to align with original data index
        participant_data.reset_index(inplace=True)

        # Find indices where 'signal' is 'Yes' (trigger events)
        trigger_indices = participant_data.index[participant_data['signal'] == 'Yes'].tolist()

        for trigger_index in trigger_indices:
            # Calculate start and end indices for the epoch
            start_idx = trigger_index - n_before
            end_idx = trigger_index + n_after

            # Handle boundary conditions
            if start_idx < 0:
                start_idx = 0
            if end_idx > len(participant_data):
                end_idx = len(participant_data)

            # Add 1 to epoch for when a section is covered by a signal

            # Get the integer position of the 'epoch' column
            epoch_col_idx = participant_data.columns.get_loc('epoch')

            # Increment the 'epoch' column for the specified range
            participant_data.iloc[start_idx:end_idx, epoch_col_idx] += 1

        # Update 'data' with the modified 'participant_data'
        data.update(participant_data.set_index('index'))

    return data

In [12]:
# USE THIS
# This method creates a datsaset which only keeps the relevant segments around signals and orders them from first to last
def segment_epochs_signal_only(data):
    data['signal'] = data['trigger'].apply(lambda x: 'Yes' if x == 1 or x == -1 else 'No')

    # Calculating time difference between samples
    fs = 250  # Sampling frequency in Hz
    time_diff = 1 / fs  # Time difference between samples

    # Time before and after trigger event (in seconds)
    time_before = -0.2
    time_after = 0.4

    # Number of samples before and after the trigger event
    n_before = int(time_before / time_diff)  # Should be 25 samples
    n_after = int(time_after / time_diff)    # Should be 175 samples

    epochs = []

    # Get the unique participant IDs
    participant_ids = data['participant_id'].unique()

    for participant_id in participant_ids:
        # Filter data for the current participant
        participant_data = data[data['participant_id'] == participant_id].reset_index(drop=True)

        # Reset epoch counter for each participant
        e = 1

        # Find indices where 'signal' is 'Yes' (trigger events)
        trigger_indices = participant_data.index[participant_data['signal'] == 'Yes'].tolist()
        one_indices = participant_data.index[participant_data['trigger'] == 1].tolist()
        minus_one_indices = participant_data.index[participant_data['trigger'] == -1].tolist()

        for trigger_index in trigger_indices:
            current_trigger = 0
            if trigger_index in one_indices:
                current_trigger = 1
            else:
                current_trigger = -1

            # Calculate start and end indices for the epoch
            start_idx = trigger_index - n_before
            end_idx = trigger_index + n_after

            # Handle boundary conditions
            if start_idx < 0:
                start_idx = 0
            if end_idx > len(participant_data):
                end_idx = len(participant_data)

            # Extract the epoch data
            epoch_data = participant_data.iloc[start_idx:end_idx].copy()

            # Assign epoch number
            epoch_data['epoch'] = e
            epoch_data['trigger'] = current_trigger
            epoch_data = epoch_data.drop('signal', axis=1)
            # Append epoch data to the list
            epochs.append(epoch_data)

            # Increment epoch counter
            e += 1

    # Combine all epochs into one DataFrame
    epochs_df = pd.concat(epochs, ignore_index=True)
    return epochs_df

In [13]:
df_epochs_signal_only = segment_epochs_signal_only(df)

In [14]:
print(df_epochs_signal_only.shape)
print(df_epochs_signal_only["epoch"].unique())
print(df_epochs_signal_only)

(300000, 13)
[   1    2    3 ... 1198 1199 1200]
        Unnamed: 0  participant_id  timestep  channel_1  channel_2  channel_3  \
0             1304               1      1305  -8.490258 -10.920556 -12.961996   
1             1305               1      1306  -7.463532  -9.866554 -10.135410   
2             1306               1      1307  -6.666172  -8.835976  -7.171571   
3             1307               1      1308  -6.265530  -7.866823  -4.506567   
4             1308               1      1309  -6.182593  -6.950725  -2.760649   
...            ...             ...       ...        ...        ...        ...   
299995       59617               5     59618  22.712175   3.506215   2.354223   
299996       59618               5     59619  20.421969   1.866547   1.300665   
299997       59619               5     59620  18.440341   0.286018  -0.325816   
299998       59620               5     59621  17.655489  -0.502610  -1.580808   
299999       59621               5     59622  18.336784  -0.

In [15]:
df_epochs_signal_only.to_csv("epoched_filted_dataset_finally.csv")